# LINC Convert — End‑to‑End Demo
This notebook is a **guided tour** of the `linc-convert` CLI, showing how to:
1) Inspect available **modalities** and **pipelines**  
2) Run a conversion (PS‑OCT example)  
3) Open the resulting **OME‑Zarr / NIfTI‑Zarr** with different backends  
4) Compare against the original MATLAB array  
5) Troubleshoot & tune performance

> **Note:** This demo is built on your local environment. Ensure `linc-convert` is installed and on your `PATH`.


## 1) Inspect CLI & Modalities
The top-level help lists **modalities** (data families) and global flags.

In [1]:
!linc-convert --help

Usage: linc-convert COMMAND

Collection of conversion scripts for LINC datasets

╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ df         Converters for Dark Field microscopy                              │
│ lsm        Converters for Light Sheet Microscopy                             │
│ psoct      Converters for PS-OCT .mat files                                  │
│ wk         Converters for Webknossos annotation                              │
│ --help -h  Display this message and exit.                                    │
│ --version  Display application version.                                      │
╰──────────────────────────────────────────────────────────────────────────────╯


Each modality contains one or more **pipelines** (subcommands).

### PS‑OCT modality
We’ll use PS‑OCT for this demo and preview the available pipelines.

In [2]:
!linc-convert psoct --help

Usage: linc-convert psoct COMMAND

Converters for PS-OCT .mat files

╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ multi_slice    Matlab to OME-Zarr.                                           │
│ single_volume  Matlab to OME-Zarr.                                           │
╰──────────────────────────────────────────────────────────────────────────────╯


Look for pipelines like `multi_slice` and `single_volume`.
For this demo, we will run **`single_volume`**

### Pipeline help: `single_volume`
Skim the options to understand input, output, chunking, sharding and additional flags.

In [3]:
!linc-convert psoct single_volume --help

Usage: linc-convert psoct single_volume [ARGS] [OPTIONS]

Matlab to OME-Zarr.                                                             

Convert OCT volumes in raw matlab files into a pyramidal OME-ZARR (or           
NIfTI-Zarr) hierarchy.

╭─ Parameters ─────────────────────────────────────────────────────────────────╮
│ *  INP --inp                         Path to the input mat file [required]   │
│    --key                             Key of the array to be extracted,       │
│                                      default to first key found              │
│    --meta                            Path to the metadata file               │
│    --orientation                     Orientation of the volume [default:     │
│                                      RAS]                                    │
│    --center --no-center              Set RAS[0, 0, 0] at FOV center          │
│                                      [default: True]                         │
│    --out                

**Key flags (quick reference):**
- `INP` / `--inp` (required): input `.mat` file
- `--key`: MATLAB variable to extract (defaults to the first key)
- `--out` / `-o`: output Zarr store path; `.nii.zarr` implies NIfTI‑Zarr layout
- `--zarr-version {2,3}`: choose Zarr 2 or 3 (v3 required for sharding)
- `--chunk`, `--shard`: control chunk/shard sizes (1 value → all dims; 3 values → `[z y x]`; 4+ → `[c z y x]`)
- `--driver`: IO backend (`zarr-python`, `tensorstore`)

## 2) Convert a Single Volume (PS‑OCT)
**Goal:** Convert a MATLAB volume to a pyramidal **NIfTI‑Zarr** (or OME‑Zarr) store.

> The cell below **runs the conversion**. Adjust `INPUT_MAT`, `MAT_KEY`, and `OUTPUT_ZARR` at the top if needed.


In [4]:
!linc-convert psoct single_volume /scratch/sample_input.mat --key Psi_ObsLSQ --out /scratch/output.nii.zarr 

2025-09-24 14:55:22,379 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (0, 0, 0) of 
2025-09-24 14:55:25,268 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (0, 1, 0) of 
2025-09-24 14:55:25,543 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (0, 2, 0) of 
2025-09-24 14:55:26,680 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (1, 0, 0) of 
2025-09-24 14:55:26,951 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (1, 1, 0) of 
2025-09-24 14:55:27,229 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (1, 2, 0) of 
2025-09-24 14:55:27,439 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (2, 0, 0) of 
2025-09-24 14:55:28,669 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (2, 1, 0) of 
2025-09-24 14:55:28,939 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (2, 2, 0) of 
2025-09-24

By default, zarr files are handled with zarr-python, we can try using tensorstore as well, which currently has much better writing performance with sharding.

In [5]:
!linc-convert psoct single_volume /scratch/sample_input.mat --key Psi_ObsLSQ --out /scratch/output-ts.nii.zarr --zarr-version 3 --shard 1024 --driver tensorstore

2025-09-24 14:55:34,234 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (0, 0, 0) of 
2025-09-24 14:55:34,321 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (0, 1, 0) of 
2025-09-24 14:55:34,423 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (0, 2, 0) of 
2025-09-24 14:55:34,498 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (1, 0, 0) of 
2025-09-24 14:55:34,608 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (1, 1, 0) of 
2025-09-24 14:55:34,686 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (1, 2, 0) of 
2025-09-24 14:55:34,735 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (2, 0, 0) of 
2025-09-24 14:55:34,824 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (2, 1, 0) of 
2025-09-24 14:55:34,915 - linc_convert.modalities.psoct.single_volume - INFO - Processing chunk (2, 2, 0) of 
2025-09-24

## 3) Compare with the original MATLAB array

In [6]:
import numpy as np
import zarr
import scipy.io as sio

# Paths & keys
INPUT_MAT = "/scratch/sample_input.mat"
MAT_KEY = "Psi_ObsLSQ"
OUT_ZARR_ZP = "/scratch/output.nii.zarr"
OUT_ZARR_TS = "/scratch/output-ts.nii.zarr"

# Load MATLAB source array
mat = sio.loadmat(INPUT_MAT)
a_mat = np.asarray(mat[MAT_KEY])

# Load Zarr arrays (zarr-python), dataset "0"
a_zp = np.asarray(zarr.open(OUT_ZARR_ZP, mode="r")["0"])
a_ts = np.asarray(zarr.open(OUT_ZARR_TS, mode="r")["0"])

# Optional: cast outputs to MATLAB dtype for strict comparison
if a_zp.dtype != a_mat.dtype:
    a_zp = a_zp.astype(a_mat.dtype, copy=False)
if a_ts.dtype != a_mat.dtype:
    a_ts = a_ts.astype(a_mat.dtype, copy=False)

# Report shapes/dtypes
print("[info] MAT   :", a_mat.shape, a_mat.dtype)
print("[info] Zarr  :", a_zp.shape, a_zp.dtype)
print("[info] TStore:", a_ts.shape, a_ts.dtype)

# Compare with NumPy testing (adjust tolerances if needed)
rtol, atol = 1e-5, 1e-8

np.testing.assert_allclose(a_mat, a_zp, rtol=rtol, atol=atol, err_msg="MAT vs output.nii.zarr differ")
print("[ok] MAT vs output.nii.zarr match within tolerances.")

np.testing.assert_allclose(a_mat, a_ts, rtol=rtol, atol=atol, err_msg="MAT vs output-ts.nii.zarr differ")
print("[ok] MAT vs output-ts.nii.zarr match within tolerances.")

[info] MAT   : (1711, 1320, 2) float64
[info] Zarr  : (1711, 1320, 2) float64
[info] TStore: (1711, 1320, 2) float64
[ok] MAT vs output.nii.zarr match within tolerances.
[ok] MAT vs output-ts.nii.zarr match within tolerances.


## Appendix — CLI Cheatsheet
```bash
# List all modalities
linc-convert --help

# List pipelines for a modality
linc-convert psoct --help

# Show detailed help for a pipeline
linc-convert psoct single_volume --help

# Minimal conversion
linc-convert psoct single_volume IN.mat -o OUT.zarr

# NIfTI-Zarr + custom key
linc-convert psoct single_volume IN.mat --key Psi_ObsLSQ -o OUT.nii.zarr

# Zarr v3 with sharding
linc-convert psoct single_volume IN.mat -o OUT.zarr --zarr-version 3 --shard 1024
```
